Data Preparation Phase

In this stage, I will be using pandas to view and explore the dataset, see what kind of columns are there and decide how I am going to create the database schema.

In [1]:
#imports
import pandas as pd

In [2]:
df = pd.read_csv("raw_dataset/listings.csv")

In [3]:
df.head(5)

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,27934,https://www.airbnb.com/rooms/27934,20250319150518,2025-03-21,city scrape,Nice room with superb city view,Our cool and comfortable one bedroom apartment...,It is very center of Bangkok and easy access t...,https://a0.muscache.com/pictures/566374/23157b...,120437,...,4.91,4.66,4.75,NaN,f,1,1,0,0,0.41
1,27979,https://www.airbnb.com/rooms/27979,20250319150518,2025-03-20,previous scrape,"Easy going landlord,easy place",NaN,NaN,https://a0.muscache.com/pictures/106247594/1d6...,120541,...,NaN,NaN,NaN,NaN,f,2,1,1,0,NaN
2,28745,https://www.airbnb.com/rooms/28745,20250319150518,2025-03-20,previous scrape,modern-style apartment in Bangkok,A modern-style apartment situated in a cool ne...,Ramkumheang neigbourhood - hip place with food...,https://a0.muscache.com/pictures/160369/92a7ec...,123784,...,NaN,NaN,NaN,NaN,f,1,0,1,0,NaN
3,47516,https://www.airbnb.com/rooms/47516,20250319150518,2025-03-20,city scrape,Beautiful waterfront house,NaN,NaN,https://a0.muscache.com/pictures/271998/b090ad...,214456,...,NaN,NaN,NaN,NaN,f,1,1,0,0,NaN
4,48736,https://www.airbnb.com/rooms/48736,20250319150518,2025-03-20,city scrape,Condo with Chaopraya River View,at a size of 25sqm for your vacation in BKK.<b...,NaN,https://a0.muscache.com/pictures/12465296/e9c0...,222005,...,5.00,5.00,5.00,NaN,f,1,0,1,0,0.01


In [4]:
df[df['neighbourhood_cleansed'] == 'Bang Sue']['longitude']

28       100.521020
127      100.521510
128      100.520820
129      100.523070
146      100.522800
            ...    
27539    100.526282
27544    100.528080
27545    100.527187
27546    100.528221
27547    100.526098
Name: longitude, Length: 515, dtype: float64

In [ ]:
df['host_name']

0            Nuttee
1               Emy
2        Familyroom
3          Anuradha
4          Athitaya
            ...    
27543      Udomchai
27544        Yangzi
27545        Yangzi
27546        Yangzi
27547        Yangzi
Name: host_name, Length: 27548, dtype: object

In [52]:
df['property_type'].unique()

array(['Entire condo', 'Private room in rental unit', 'Entire home',
       'Private room in serviced apartment', 'Room in hostel',
       'Private room in home', 'Entire rental unit',
       'Private room in hostel', 'Entire guesthouse',
       'Entire serviced apartment', 'Room in aparthotel',
       'Room in serviced apartment', 'Private room in condo',
       'Entire loft', 'Private room in guesthouse', 'Entire villa',
       'Private room in townhouse', 'Room in boutique hotel',
       'Shared room in hostel', 'Room in hotel', 'Private room in loft',
       'Private room', 'Entire townhouse', 'Private room in cabin',
       'Private room in bed and breakfast',
       'Shared room in bed and breakfast', 'Private room in island',
       'Private room in villa', 'Entire cottage', 'Shared room in home',
       'Entire chalet', 'Private room in guest suite', 'Entire cabin',
       'Entire place', 'Entire guest suite', 'Room in bed and breakfast',
       'Private room in boat', 'Entire 

In [57]:
df['review_scores_communication'].isna().sum()

np.int64(9467)

In [6]:
df.columns

Index(['id', 'listing_url', 'scrape_id', 'last_scraped', 'source', 'name',
       'description', 'neighborhood_overview', 'picture_url', 'host_id',
       'host_url', 'host_name', 'host_since', 'host_location', 'host_about',
       'host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'host_is_superhost', 'host_thumbnail_url', 'host_picture_url',
       'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'neighbourhood',
       'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'latitude',
       'longitude', 'property_type', 'room_type', 'accommodates', 'bathrooms',
       'bathrooms_text', 'bedrooms', 'beds', 'amenities', 'price',
       'minimum_nights', 'maximum_nights', 'minimum_minimum_nights',
       'maximum_minimum_nights', 'minimum_maximum_nights',
       'maximum_maximum_nights', 'minimum_nights_avg_ntm',
       'maximum_nights_avg_ntm', 'ca

In [44]:
df['host_id']

0           120437
1           120541
2           123784
3           214456
4           222005
           ...    
27543    679097032
27544    653148266
27545    653148266
27546    653148266
27547    653148266
Name: host_id, Length: 27548, dtype: int64

In [48]:
df.loc[df['host_id'] == 653148266]['host_since']

22375    2024-09-20
22423    2024-09-20
22426    2024-09-20
22438    2024-09-20
22439    2024-09-20
            ...    
27057    2024-09-20
27544    2024-09-20
27545    2024-09-20
27546    2024-09-20
27547    2024-09-20
Name: host_since, Length: 140, dtype: object

In [41]:
cols = ["id", "last_scraped", "name", "description", "host_since", "host_is_superhost", "host_acceptance_rate", "host_response_rate", "neighbourhood_cleansed"]

In [32]:
df.head(5)

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,27934,https://www.airbnb.com/rooms/27934,20250319150518,2025-03-21,city scrape,Nice room with superb city view,Our cool and comfortable one bedroom apartment...,It is very center of Bangkok and easy access t...,https://a0.muscache.com/pictures/566374/23157b...,120437,...,4.91,4.66,4.75,NaN,f,1,1,0,0,0.41
1,27979,https://www.airbnb.com/rooms/27979,20250319150518,2025-03-20,previous scrape,"Easy going landlord,easy place",NaN,NaN,https://a0.muscache.com/pictures/106247594/1d6...,120541,...,NaN,NaN,NaN,NaN,f,2,1,1,0,NaN
2,28745,https://www.airbnb.com/rooms/28745,20250319150518,2025-03-20,previous scrape,modern-style apartment in Bangkok,A modern-style apartment situated in a cool ne...,Ramkumheang neigbourhood - hip place with food...,https://a0.muscache.com/pictures/160369/92a7ec...,123784,...,NaN,NaN,NaN,NaN,f,1,0,1,0,NaN
3,47516,https://www.airbnb.com/rooms/47516,20250319150518,2025-03-20,city scrape,Beautiful waterfront house,NaN,NaN,https://a0.muscache.com/pictures/271998/b090ad...,214456,...,NaN,NaN,NaN,NaN,f,1,1,0,0,NaN
4,48736,https://www.airbnb.com/rooms/48736,20250319150518,2025-03-20,city scrape,Condo with Chaopraya River View,at a size of 25sqm for your vacation in BKK.<b...,NaN,https://a0.muscache.com/pictures/12465296/e9c0...,222005,...,5.00,5.00,5.00,NaN,f,1,0,1,0,0.01
